# .s2p Reader

In [188]:
import os
import numpy as np
import skrf as skrf

#Calculo los parámetros S para una frecuencia determinada en cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters= {}
    
    # Se convierte la frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p= [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo= os.path.join(carpeta, archivo_s2p)
            red= skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia= np.argmin(np.abs(red.f - frecuencia))
            print(indice_frecuencia)
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

## Cálculo del determinante $\Delta$

In [189]:
def calcular_delta(parametros_s):
    deltas= {}
    for archivo, parametros in parametros_s.items():
        s11= parametros[0, 0]
        s21= parametros[1, 0]
        s12= parametros[0, 1]
        s22= parametros[1, 1]

        dt= s11*s22 - s12*s21
        print(dt)
        deltas[archivo]= dt
    return deltas

## Cálculo de Factor de Rollet $K$

In [190]:
def calcular_k(s_parametros, deltas):
    ks= {}
    polarizacionk_incond_est= {}
    
    for archivo, parametros in s_parametros.items():
        s11= parametros[0, 0]
        s12= parametros[0, 1]
        s21= parametros[1, 0]
        s22= parametros[1, 1]
        
        #abs_delta= abs(deltas[archivo])

        K= (1- abs(s11)**2 - abs(s22)**2 + abs(deltas[archivo])**2)/(2*abs(s12*s21))

        ks[archivo]= K
        
        if K > 1:
            polarizacionk_incond_est[archivo]= {'K': K}

    return ks, polarizacionk_incond_est

## Interfaz

In [191]:
frecuencia= 1.6
Zo= 50
carpetas= ["Datasheets"]
#ic_filter= 10

s_parameters= calcular_parametros_s(frecuencia, carpetas)
print(s_parameters)
#deltas= calcular_delta(s_parameters)
#ks, polarizacionk_incond_est= calcular_k(s_parameters, deltas)


30
{'BFP640_VCE_1.5V_IC_22mA.s2p': array([[-0.28714724 -0.11195403j,  0.02849955 +0.03837468j],
       [ 1.3055463 +12.85286388j,  0.12410209 -0.24147655j]])}
